In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import regexp_extract, col, input_file_name

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,mysql:mysql-connector-java:8.0.33')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [32]:
schema = StructType([
    StructField("data", StringType())
])

df = spark.read.schema(schema).json('s3a://bucket-biosentinel-2semestre-client/umid-temp/2024-09-11/*.json')

df = df.withColumn("fk_sensor", regexp_extract(input_file_name(), r'.*_(\d+)\.json', 1).cast(IntegerType()))
df = df.withColumn("value", col("data"))
df = df.withColumn("created_at", regexp_extract(input_file_name(), r'umid-temp_(\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})_(\d+)\.json', 1))

# Selecione as colunas necessárias para o CSV
df_csv = df.select("fk_sensor", "value", "created_at")

# Exiba o DataFrame como um CSV
df_csv.show()

+---------+--------------------+-------------------+
|fk_sensor|               value|         created_at|
+---------+--------------------+-------------------+
|        5|{"value":{"humid"...|2024-09-11_00-10-00|
|        5|{"value":{"humid"...|2024-09-11_00-20-00|
|        5|{"value":{"humid"...|2024-09-11_00-30-00|
|        5|{"value":{"humid"...|2024-09-11_00-40-00|
|        5|{"value":{"humid"...|2024-09-11_01-00-00|
|        5|{"value":{"humid"...|2024-09-11_01-10-00|
|        5|{"value":{"humid"...|2024-09-11_01-30-00|
|        5|{"value":{"humid"...|2024-09-11_01-50-00|
|        5|{"value":{"humid"...|2024-09-11_02-00-00|
|        5|{"value":{"humid"...|2024-09-11_02-30-00|
|        5|{"value":{"humid"...|2024-09-11_02-40-00|
|        5|{"value":{"humid"...|2024-09-11_03-00-00|
|        5|{"value":{"humid"...|2024-09-11_03-10-00|
|        5|{"value":{"humid"...|2024-09-11_03-20-00|
|        5|{"value":{"humid"...|2024-09-11_03-30-00|
|        5|{"value":{"humid"...|2024-09-11_03-

In [33]:
# Conexão com o banco
url = 'jdbc:mysql://database-biosentinel.c2xugs21oous.us-east-1.rds.amazonaws.com:3306/biosentinel-database'
tablename = 'data_sensor'
user = 'biosentineladmin'
password = '4d`#NT3wgwQb'

df_csv \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.mysql.cj.jdbc.Driver") \
  .option("url", url) \
  .option("dbtable", tablename) \
  .option("user", user) \
  .option("password", password) \
  .save()

print('done')

done
